# Session Generator Without Fine-tuning

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import re
import ast
from utils import load_pickle
from sentence_transformers import SentenceTransformer, util

/home/guest/r13946024/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Part I Load model

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.61s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [4]:
def extract_product_name(raw_output, fallback="Unknown Product"):
    # Split the response into lines
    lines = raw_output.strip().split("\n")

    # Find the last non-empty line
    for line in reversed(lines):
        clean_line = line.strip()
        if clean_line:
            # Remove common prefixes
            for prefix in ["Sure,", "Here is", "Here are", "Of course,", "I'd be happy to help!"]:
                if clean_line.lower().startswith(prefix.lower()):
                    clean_line = clean_line[len(prefix):].strip()
            # Remove category hints if present
            clean_line = re.sub(r'for the ".+?" category', "", clean_line).strip()
            
            # Return the cleaned line if it looks like a product name
            if clean_line and len(clean_line.split()) >= 2:
                return clean_line

    # Fallback if no valid name is found
    print(f"⚠️ No valid product name found in: '{raw_output}'")
    return fallback

# === Extract a Python List from LLM Output ===
def extract_list_from_text(text, fallback=None):
    if not text or not isinstance(text, str):
        print("⚠️ Invalid input. Using fallback.")
        return fallback or []

    # Strip leading instructions and whitespace
    cleaned_text = text.strip()

    # Extract the first valid list
    match = re.search(r'\[\s*(?:[^\[\]]|\[[^\[\]]*\])*\s*\]', cleaned_text, re.DOTALL)
    if match:
        try:
            extracted = match.group(0).replace("\n", "").replace("    ", "").strip()
            # Basic sanity check to avoid explanation leakage
            if extracted.startswith("[") and extracted.endswith("]"):
                return ast.literal_eval(extracted)
            else:
                print("⚠️ Unexpected non-list format. Using fallback.")
        except Exception as e:
            print(f"⚠️ Parsing failed: {e}")
    
    print("⚠️ No valid list found. Using fallback.")
    return fallback or []

# Part II Functions & Prompts



## prompt #1: Get Categories form History Sessions

In [5]:
def get_categories_from_history(history_str, category_list, n=9):
    categories_str = ", ".join(category_list)
    # Define the system and user prompts
    system_prompt = "You are a fashion recommendation assistant."
    user_prompt = f"""
    Given this purchase history:
    {history_str}

    Select exactly {n} categories from the following list:
    {categories_str}

    ⚠️ Respond **only** with a Python list of exactly {n} categories, **without any additional text, punctuation, or explanations.**

    ✅ Format exactly like this:
    ["Category1", "Category2", "Category3", ..., "Category{n}"]

    ❗ Do not add any introductory words, explanations, or context.
    ❗ Do not include line breaks within the list.
    ❗ Do not add extra punctuation or bullet points.
    """

    prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt} [/INST]"
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    response = response.split("[/INST]")[-1].strip()
    return response








## Prompt #2: Generate New Session from Categories

In [6]:
def get_item_from_category(category):
    # Define the system and user prompts
    system_prompt = "You are a fashion product naming assistant."
    user_prompt = f"""
    Generate a single, catchy, and realistic product name for the "{category}" category.

    ⚠️ **Output Requirements:**  
    - Respond with only the product name as a single line of text.  
    - Do not include quotes, bullet points, or explanations.  
    - Use realistic, catchy names like "Spanx Alot Swimsuit" or "Jerry Jogger Bottoms".

    ❗ **Examples:**  
    Slim-fit T-shirt  
    Leather Jacket  
    Spanx Alot Swimsuit  
    """

    # Construct the full prompt
    full_prompt = f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_prompt.strip()} [/INST]"

    # Generate the response
    inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)

    # Decode and return the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    response = response.split("[/INST]")[-1].strip()
    return response


### Utils Function

- **prefilter_categories(items_list, category_list, top_k=20)**

    此函式用於在進入 LLM 前，根據使用者歷史購買商品所對應的 product type（商品類別），從所有可選類別中預先篩選出最相關的前 top_k 個類別，避免因為類別數量過多導致超出 LLM 的上下文長度限制（context window overflow）。

- **article_to_product_type(article_id)**

    此函式將 articleId 轉換成相對應的 product type (category)

In [7]:
# === Pre-Filter Categories with Embeddings ===
def prefilter_categories(items_list, category_list, top_k=20):
    model = SentenceTransformer("all-MiniLM-L6-v2")
    
    # Encode the history as a single vector
    history_embedding = model.encode(items_list, convert_to_tensor=True)
    
    # Encode all categories
    category_embeddings = model.encode(category_list, convert_to_tensor=True)
    
    # Compute similarity
    cos_scores = util.cos_sim(history_embedding, category_embeddings)[0]
    top_results = cos_scores.topk(k=top_k)
    
    # Extract top-K categories
    top_categories = [category_list[i] for i in top_results.indices]
    top_scores = [cos_scores[i].item() for i in top_results.indices]
    
    # Return as a dictionary for better interpretability
    return top_categories


def article_to_product_type(article_id):
    mapping = load_pickle("data/article_to_product_mapping.pkl")
    return mapping.get(article_id, "Unknown Product Type")

## Main Pipeline

**enrich_user_session(user_sessions, category_list, n=9, max_history_length=10)**

這個函式是整個推薦系統流程的核心，它的目的在於將原本的使用者購買紀錄 user session 擴充為更完整、更擬真的購物序列，藉由語意相似度 + LLM 輔助推理生成更多合理的商品。

In [8]:
# === Generate a New Enriched User Session ===
def enrich_user_session(user_sessions, category_list, n=9, max_history_length=10):
    session = {uid: items.copy() for uid, items in user_sessions.items()}

    for uid, items in session.items():
        # Convert item IDs to product names
        history_str = " ".join([article_to_product_type(item) for item in items])
        print(f"User: {uid}, History: {history_str}\n")
        top_categories = prefilter_categories(history_str, category_list, top_k=20)
        print(f"Top categories: {top_categories}\n")
        # Generate Categories
        raw_output = get_categories_from_history(history_str, top_categories)
        print("LLM raw output:", raw_output, "\n")
        print("End of the output")
        categories = extract_list_from_text(raw_output, fallback=["tops", "accessories", "shoes"])
        print(f"Selected Categories: {categories}\n")

        # Generate Items for Each Category
        while len(items) < max_history_length:
            for category in categories:
                raw_item = get_item_from_category(category)
                print("Raw item LLM output: \n", raw_item)
                item = extract_product_name(raw_item, fallback="Unkown")
                print("Extracted: ", item)
                if len(items) > 0:
                    items.insert(-1, item)
                else:
                    # If the list is empty, just append
                    items.append(item)
                if len(items) >= max_history_length:
                    break

    return session

# Part III Implementation

## Load pickle file

##  Load Category List

In [ ]:
import pandas as pd

categories_df = pd.read_csv("product_types.csv")
category_list = categories_df["category_name"].tolist()
print("Total categories loaded:", len(category_list))
print("Sample categories:", category_list[:10])


Total categories loaded: 131
Sample categories: ['Vest top', 'Bra', 'Underwear Tights', 'Socks', 'Leggings/Tights', 'Sweater', 'Top', 'Trousers', 'Hair clip', 'Umbrella']


##  Test Category Prefiltering

## Test Session Enrichment

This block is only for demonstration, not real data.

In [ ]:
import time 

start_time = time.time()
user_session = {
    "00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657": [841260003, 887593002, 890498002, 795440001, 859416011, 568601043],
    "0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa": [759191008, 800436010, 814686001, 590928022, 698276006, 749699013],
}

new_session = enrich_user_session(user_session, category_list, n=3, max_history_length=9)
print("Enriched Session:", new_session)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"執行時間：{elapsed_time:.4f} 秒")

User: 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657, History: T-shirt Top Jacket Vest top Sweater Blazer

Top categories: ['Vest top', 'T-shirt', 'Sweater', 'Blazer', 'Jacket', 'Tailored Waistcoat', 'Swimwear top', 'Hoodie', 'Shirt', 'Garment Set', 'Outdoor Waistcoat', 'Polo shirt', 'Bodysuit', 'Bikini top', 'Swimsuit', 'Swimwear bottom', 'Swimwear set', 'Clothing mist', 'Zipper head', 'Robe']

LLM raw output: Sure, here are the 9 categories based on the purchase history:

["Vest top", "T-shirt", "Sweater", "Blazer", "Jacket", "Tailored Waistcoat", "Swimwear top", "Hoodie", "Shirt"] 

End of the output
Selected Categories: ['Vest top', 'T-shirt', 'Sweater', 'Blazer', 'Jacket', 'Tailored Waistcoat', 'Swimwear top', 'Hoodie', 'Shirt']

Raw item LLM output: 
 Sure, here's a product name for the "Vest top" category:

FlexFit Vest
Extracted:  FlexFit Vest
Raw item LLM output: 
 FreshFit Tee
Extracted:  FreshFit Tee
Raw item LLM output: 
 Sure, here's a product name for th